In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [17]:
# Загрузка данных
data = pd.read_csv('/home/eva/project/predictive_real_estate_price_analytics_service/data/progress/progress_data.csv')
data.head()

,url_id,total_meters,price
0,316749660,38.9,13477999
1,315778606,32.0,18700000
2,311419733,34.1,19000000
3,315916888,32.9,8700000
4,306867070,41.1,33000000


In [18]:
'''
# Стандартизация признака
#scaler = StandardScaler()
#data['total_meters'] = scaler.fit_transform(data[['total_meters']])

# Разделение на признаки и целевую переменную
X = data[['total_meters']]  # только один признак - площадь
y = data['price']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
# todo: use logging
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

# Коэффициенты модели
print(f"Коэффициент при площади: {model.coef_[0]:.2f}")
print(f"Свободный член: {model.intercept_:.2f}")
'''

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Разделение на признаки и целевую переменную
X = data[['total_meters']]  # только один признак - площадь
y = data['price']

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Создание и обучение модели CatBoost
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    verbose=100,  # выводит информацию каждые 100 итераций
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

# Для CatBoost коэффициенты модели интерпретируются иначе, чем в линейной регрессии
# Можно получить важность признаков
print("\nВажность признаков:")
for name, score in zip(X.columns, model.get_feature_importance()):
    print(f"{name}: {score:.2f}")

0:	learn: 106633057.9008838	total: 991us	remaining: 990ms
100:	learn: 65453596.6870823	total: 109ms	remaining: 970ms
200:	learn: 63402118.8485576	total: 198ms	remaining: 789ms
300:	learn: 62093891.4756775	total: 291ms	remaining: 676ms
400:	learn: 61479929.6243515	total: 382ms	remaining: 571ms
500:	learn: 61108457.6560380	total: 471ms	remaining: 469ms
600:	learn: 60892166.0870342	total: 557ms	remaining: 370ms
700:	learn: 60746820.2583275	total: 650ms	remaining: 277ms
800:	learn: 60646330.8382143	total: 737ms	remaining: 183ms
900:	learn: 60574170.5977732	total: 823ms	remaining: 90.4ms
999:	learn: 60519768.7623818	total: 919ms	remaining: 0us
Среднеквадратичная ошибка (MSE): 7372181929576480.00
Корень из среднеквадратичной ошибки (RMSE): 85861411.18
Коэффициент детерминации R²: 0.525939
Средняя ошибка предсказания: 37593530.69 рублей

Важность признаков:
total_meters: 100.00


In [19]:
import joblib

# Сохранение модели
model_path = '/home/eva/project/predictive_real_estate_price_analytics_service/models/linear_regression_model.pkl'

joblib.dump(model, model_path)
print(f"Модель сохранена в файл {model_path}") 

# Загрузка модели
loaded_model = joblib.load(model_path)
print("Модель загружена из файла")

Модель сохранена в файл /home/eva/project/predictive_real_estate_price_analytics_service/models/linear_regression_model.pkl
Модель загружена из файла
